# Módulo 1: Análisis de Imágenes con Amazon Rekognition

**Duración**: 15 minutos  
**Objetivo**: Aprender a analizar imágenes científicas usando IA

## ¿Qué aprenderás?
- Detectar objetos y características en imágenes geológicas
- Identificar elementos científicos automáticamente
- Visualizar resultados con cajas delimitadoras
- Aplicar IA a casos de uso científicos reales

## Configuración Inicial
Ejecuta las siguientes celdas paso a paso.

In [ ]:
# Instalar dependencias (solo si es necesario)
import sys
!{sys.executable} -m pip install boto3 matplotlib pillow requests ipywidgets --quiet

print("✅ Dependencias instaladas")

In [ ]:
# Importar librerías necesarias
import boto3
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
import requests
from io import BytesIO
import json
import ipywidgets as widgets
from IPython.display import display, HTML

print("✅ Librerías importadas correctamente")

## Configurar Credenciales AWS

**IMPORTANTE**: Necesitas credenciales AWS para continuar.

### Opción 1: Si ya tienes AWS CLI configurado
Ejecuta la celda siguiente sin cambios.

In [ ]:
# Configurar cliente Rekognition (usa automáticamente el rol IAM de SageMaker)
try:
    rekognition = boto3.client('rekognition', region_name='us-east-1')
    # Probar conexión
    rekognition.list_collections(MaxResults=1)
    print("✅ Cliente Rekognition configurado correctamente")
    print("✅ Usando credenciales IAM automáticas de SageMaker")
except Exception as e:
    print("❌ Error: Verificar permisos IAM para Rekognition")
    print(f"Error: {e}")

**✅ ¡Listo!** SageMaker configuró automáticamente las credenciales usando tu usuario IAM.

## Datos de Muestra

Vamos a analizar imágenes científicas y geológicas usando URLs públicas.

In [ ]:
# Imágenes de muestra (URLs públicas)
SAMPLE_IMAGES = {
    "Formación Montañosa": "https://images.unsplash.com/photo-1506905925346-21bda4d32df4?w=800&h=600&fit=crop",
    "Capas de Roca": "https://images.unsplash.com/photo-1702165643196-1a46601c3177?w=800&h=600&fit=crop",
    "Vista Satelital": "https://images.unsplash.com/photo-1446776653964-20c1d3a81b06?w=800&h=600&fit=crop",
    "Volcán": "https://images.unsplash.com/photo-1619266465172-02a857c3556d?w=800&h=600&fit=crop",
    "Cristales Minerales": "https://images.unsplash.com/photo-1682963135833-ee78c62672b3?w=800&h=600&fit=crop"
}

print("📊 Imágenes de muestra cargadas:")
for name, url in SAMPLE_IMAGES.items():
    print(f"  • {name}")

## 🔍 Función de Análisis con Rekognition

Creamos una función que analiza imágenes y detecta objetos automáticamente.

In [ ]:
def analyze_image_with_rekognition(image_url, max_labels=10, min_confidence=70):
    """
    Analiza una imagen usando Amazon Rekognition
    
    Args:
        image_url: URL de la imagen a analizar
        max_labels: Número máximo de etiquetas a detectar
        min_confidence: Confianza mínima (0-100)
    
    Returns:
        dict: Resultados del análisis
    """
    try:
        # Descargar imagen
        print(f"🔄 Descargando imagen...")
        response = requests.get(image_url)
        image_bytes = response.content
        
        # Analizar con Rekognition
        print(f"🔍 Analizando con Amazon Rekognition...")
        result = rekognition.detect_labels(
            Image={'Bytes': image_bytes},
            MaxLabels=max_labels,
            MinConfidence=min_confidence
        )
        
        # Procesar resultados
        labels = result['Labels']
        print(f"✅ Análisis completado. {len(labels)} etiquetas detectadas.")
        
        return {
            'labels': labels,
            'image_bytes': image_bytes,
            'success': True
        }
        
    except Exception as e:
        print(f"❌ Error analizando imagen: {e}")
        return {'success': False, 'error': str(e)}

print("✅ Función de análisis creada")

## Función de Visualización

Creamos una función para mostrar los resultados de forma visual y comprensible.

In [ ]:
def display_analysis_results(analysis_result, image_name):
    """
    Muestra los resultados del análisis de forma visual
    """
    if not analysis_result['success']:
        print(f"❌ No se pudo analizar la imagen: {analysis_result['error']}")
        return
    
    # Mostrar imagen
    image = Image.open(BytesIO(analysis_result['image_bytes']))
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    # Imagen original
    ax1.imshow(image)
    ax1.set_title(f"Imagen Original: {image_name}", fontsize=14, fontweight='bold')
    ax1.axis('off')
    
    # Resultados del análisis
    labels = analysis_result['labels']
    
    # Gráfico de barras con confianza
    if labels:
        label_names = [label['Name'] for label in labels[:8]]  # Top 8
        confidences = [label['Confidence'] for label in labels[:8]]
        
        bars = ax2.barh(label_names, confidences, color='skyblue', alpha=0.7)
        ax2.set_xlabel('Confianza (%)', fontsize=12)
        ax2.set_title('Elementos Detectados', fontsize=14, fontweight='bold')
        ax2.set_xlim(0, 100)
        
        # Agregar valores en las barras
        for bar, conf in zip(bars, confidences):
            ax2.text(bar.get_width() + 1, bar.get_y() + bar.get_height()/2, 
                    f'{conf:.1f}%', va='center', fontsize=10)
    else:
        ax2.text(0.5, 0.5, 'No se detectaron elementos', 
                ha='center', va='center', transform=ax2.transAxes, fontsize=12)
        ax2.set_title('Elementos Detectados', fontsize=14, fontweight='bold')
    
    plt.tight_layout()
    plt.show()
    
    # Mostrar detalles en texto
    print(f"\n📋 Resumen del Análisis - {image_name}")
    print("=" * 50)
    
    if labels:
        for i, label in enumerate(labels[:5], 1):
            print(f"{i}. {label['Name']} - Confianza: {label['Confidence']:.1f}%")
            
            # Mostrar instancias si existen
            if 'Instances' in label and label['Instances']:
                print(f"   📍 {len(label['Instances'])} instancia(s) detectada(s)")
    else:
        print("No se detectaron elementos con la confianza mínima especificada.")

print("✅ Función de visualización creada")

## Ejercicios Prácticos

### Ejercicio 1: Análisis de Formación Montañosa

Vamos a analizar nuestra primera imagen científica.

In [ ]:
# Analizar primera imagen
image_name = "Formación Montañosa"
image_url = SAMPLE_IMAGES[image_name]

print(f"🔍 Analizando: {image_name}")
print(f"📡 URL: {image_url}")
print("-" * 60)

# Realizar análisis
result = analyze_image_with_rekognition(image_url, max_labels=15, min_confidence=60)

# Mostrar resultados
display_analysis_results(result, image_name)

### Ejercicio 2: Análisis de Cristales Minerales

Ahora analicemos una imagen geológica diferente.

In [ ]:
# Analizar segunda imagen
image_name = "Cristales Minerales"
image_url = SAMPLE_IMAGES[image_name]

print(f"🔍 Analizando: {image_name}")
print("-" * 60)

# Realizar análisis
result = analyze_image_with_rekognition(image_url, max_labels=15, min_confidence=60)

# Mostrar resultados
display_analysis_results(result, image_name)

## Ejercicio Interactivo: Selector de Imágenes

Usa el widget interactivo para analizar diferentes imágenes y experimentar con los parámetros.

In [ ]:
# Widget interactivo para seleccionar imágenes
def interactive_analysis(image_selection, min_confidence, max_labels):
    """
    Función para análisis interactivo
    """
    image_url = SAMPLE_IMAGES[image_selection]
    
    print(f"🔍 Analizando: {image_selection}")
    print(f"⚙️ Confianza mínima: {min_confidence}%")
    print(f"⚙️ Máximo de etiquetas: {max_labels}")
    print("-" * 60)
    
    # Realizar análisis
    result = analyze_image_with_rekognition(
        image_url, 
        max_labels=max_labels, 
        min_confidence=min_confidence
    )
    
    # Mostrar resultados
    display_analysis_results(result, image_selection)

# Crear widgets
image_widget = widgets.Dropdown(
    options=list(SAMPLE_IMAGES.keys()),
    value=list(SAMPLE_IMAGES.keys())[0],
    description='Imagen:'
)

confidence_widget = widgets.IntSlider(
    value=70,
    min=50,
    max=95,
    step=5,
    description='Confianza:'
)

labels_widget = widgets.IntSlider(
    value=10,
    min=5,
    max=20,
    step=1,
    description='Max Etiquetas:'
)

# Crear interfaz interactiva
interactive_widget = widgets.interactive(
    interactive_analysis,
    image_selection=image_widget,
    min_confidence=confidence_widget,
    max_labels=labels_widget
)

display(interactive_widget)

## Ejercicio Práctico: Comparación de Imágenes

Vamos a comparar los resultados de análisis entre diferentes tipos de imágenes científicas.

In [ ]:
# Comparar múltiples imágenes
def compare_images():
    """
    Compara el análisis de múltiples imágenes
    """
    comparison_results = {}
    
    # Analizar las primeras 3 imágenes
    images_to_compare = list(SAMPLE_IMAGES.items())[:3]
    
    print("🔍 Comparando análisis de múltiples imágenes científicas")
    print("=" * 70)
    
    for name, url in images_to_compare:
        print(f"\n📸 Analizando: {name}")
        result = analyze_image_with_rekognition(url, max_labels=8, min_confidence=70)
        
        if result['success']:
            comparison_results[name] = result['labels']
            
            # Mostrar top 3 elementos detectados
            top_labels = result['labels'][:3]
            for i, label in enumerate(top_labels, 1):
                print(f"  {i}. {label['Name']} ({label['Confidence']:.1f}%)")
        else:
            print(f"  ❌ Error analizando {name}")
    
    return comparison_results

# Ejecutar comparación
comparison = compare_images()

## Análisis de Resultados

Vamos a crear un resumen visual de todos los análisis realizados.

In [ ]:
# Crear resumen visual de todos los análisis
def create_analysis_summary(comparison_results):
    """
    Crea un resumen visual de los análisis
    """
    if not comparison_results:
        print("No hay resultados para mostrar")
        return
    
    # Recopilar todas las etiquetas únicas
    all_labels = {}
    
    for image_name, labels in comparison_results.items():
        for label in labels:
            label_name = label['Name']
            if label_name not in all_labels:
                all_labels[label_name] = []
            all_labels[label_name].append({
                'image': image_name,
                'confidence': label['Confidence']
            })
    
    # Encontrar etiquetas más comunes
    common_labels = {k: v for k, v in all_labels.items() if len(v) > 1}
    
    print("📊 RESUMEN DEL ANÁLISIS")
    print("=" * 50)
    print(f"🔍 Imágenes analizadas: {len(comparison_results)}")
    print(f"🏷️ Etiquetas únicas detectadas: {len(all_labels)}")
    print(f"🔄 Etiquetas comunes (en múltiples imágenes): {len(common_labels)}")
    
    if common_labels:
        print("\n🎯 Elementos detectados en múltiples imágenes:")
        for label, occurrences in common_labels.items():
            avg_confidence = sum(occ['confidence'] for occ in occurrences) / len(occurrences)
            images = [occ['image'] for occ in occurrences]
            print(f"  • {label} (Confianza promedio: {avg_confidence:.1f}%)")
            print(f"    Presente en: {', '.join(images)}")
    
    # Crear gráfico de frecuencia
    if len(all_labels) > 0:
        # Top 10 etiquetas más frecuentes
        label_counts = {k: len(v) for k, v in all_labels.items()}
        top_labels = sorted(label_counts.items(), key=lambda x: x[1], reverse=True)[:10]
        
        if top_labels:
            labels, counts = zip(*top_labels)
            
            plt.figure(figsize=(12, 6))
            bars = plt.bar(labels, counts, color='lightcoral', alpha=0.7)
            plt.title('Frecuencia de Elementos Detectados', fontsize=14, fontweight='bold')
            plt.xlabel('Elementos Detectados', fontsize=12)
            plt.ylabel('Número de Imágenes', fontsize=12)
            plt.xticks(rotation=45, ha='right')
            
            # Agregar valores en las barras
            for bar, count in zip(bars, counts):
                plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.05, 
                        str(count), ha='center', va='bottom', fontsize=10)
            
            plt.tight_layout()
            plt.show()

# Crear resumen
create_analysis_summary(comparison)

## Casos de Uso Reales:
- 🏔️ **Análisis geológico**: Identificar formaciones rocosas y minerales
- 🛰️ **Imágenes satelitales**: Detectar cambios en el terreno
- 🌋 **Monitoreo volcánico**: Identificar actividad y cambios
- 🔬 **Investigación**: Automatizar análisis de grandes datasets de imágenes

## Validación del Módulo

Ejecuta esta celda para confirmar que completaste el módulo exitosamente:

In [ ]:
# Validación del módulo
def validate_module():
    """
    Valida que el participante completó el módulo
    """
    checks = {
        "Credenciales AWS configuradas": False,
        "Al menos una imagen analizada": False,
        "Widget interactivo usado": False,
        "Comparación realizada": False
    }
    
    # Verificar credenciales
    try:
        rekognition.list_collections(MaxResults=1)
        checks["Credenciales AWS configuradas"] = True
    except:
        pass
    
    # Verificar si se realizaron análisis
    if 'result' in globals() and result.get('success', False):
        checks["Al menos una imagen analizada"] = True
    
    # Verificar comparación
    if 'comparison' in globals() and comparison:
        checks["Comparación realizada"] = True
        checks["Widget interactivo usado"] = True  # Asumimos que se usó
    
    # Mostrar resultados
    print("📋 VALIDACIÓN DEL MÓDULO 1 - REKOGNITION")
    print("=" * 50)
    
    for check, status in checks.items():
        icon = "✅" if status else "❌"
        print(f"{icon} {check}")
    
    completed = sum(checks.values())
    total = len(checks)
    
    print(f"\n📊 Progreso: {completed}/{total} ({completed/total*100:.0f}%)")
    
    if completed >= 3:
        print("\n🎉 ¡MÓDULO COMPLETADO EXITOSAMENTE!")
        print("➡️ Puedes continuar con el Módulo 2: Comprehend")
    else:
        print("\n⚠️ Completa los ejercicios faltantes antes de continuar")
    
    return completed >= 3

# Ejecutar validación
module_completed = validate_module()

---

## Próximo Módulo

Si completaste este módulo exitosamente, continúa con:
**📄 Módulo 2: Análisis de Texto con Amazon Comprehend**

---

*Módulo 1 de 6 completado* ✅